In [ ]:
import pandas as pd

# install Pint if necessary
try:
    from pint import UnitRegistry
except ImportError:
    !pip install pint

In [ ]:
# download modsim.py if necessary

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'modsim.py')

In [ ]:
# Menghapus kolom yang tidak diperlukan
data_have_index = [
    "sample/anxiety-disorders-males-vs-females.csv",
    "sample/bipolar-disorder-in-males-vs-females.csv",
    "sample/depression-males-vs-females.csv",
    "sample/eating-disorders-in-males-vs-females.csv",
    "sample/Mental health Depression disorder Data.csv",
    "sample/schizophrenia-in-males-vs-females.csv"
]

for file_path in data_have_index:
    df = pd.read_csv(file_path)
    if 'index' in df.columns:
        df = df.drop('index', axis=1)
    df.to_csv(file_path, index=False)

In [ ]:
df_grouped_bundir = pd.read_csv("sample/suicide-rates-among-young-people.csv")
df_grouped_bundir = df_grouped_bundir.groupby('Year').mean().reset_index()

In [ ]:
df_grouped_bundir.columns

In [ ]:
df_grouped_bundir['Change_15_19'] = df_grouped_bundir['Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19'].round()
df_grouped_bundir['Change_20_24'] = df_grouped_bundir['Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24'].round()
df_grouped_bundir['Change_25_34'] = df_grouped_bundir['Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34'].round()

print(df_grouped_bundir[['Year', 'Change_15_19', 'Change_20_24', 'Change_25_34']])

In [ ]:
df_grouped_bundir['Mean_Total'] = df_grouped_bundir[['Change_15_19', 'Change_20_24', 'Change_25_34']].sum(axis=1).round()

print(df_grouped_bundir[['Year', 'Change_15_19', 'Change_20_24', 'Change_25_34', 'Mean_Total']])

In [ ]:
# Anxiety
df_anxiety = pd.read_csv("sample/anxiety-disorders-males-vs-females.csv")
# df_anxiety.head(20000)
# df_anxiety.columns

df_anxiety_filtered = df_anxiety[(df_anxiety['Year'] >= 2000) & (df_anxiety['Year'] <= 2019)]

df_anxiety_mean = df_anxiety_filtered.groupby('Year').agg({
    'Prevalence - Anxiety disorders - Sex: Male - Age: Age-standardized (Percent)': 'mean',
    'Prevalence - Anxiety disorders - Sex: Female - Age: Age-standardized (Percent)': 'mean'
})

total_df_anxiety_mean =  df_anxiety_mean['Prevalence - Anxiety disorders - Sex: Female - Age: Age-standardized (Percent)'] + df_anxiety_mean['Prevalence - Anxiety disorders - Sex: Male - Age: Age-standardized (Percent)']

total_df_anxiety_mean.round()
